# Trabajo Práctico 1 - Matrices de Insumo Producto

## Enunciado

### Consigna 1 - 

Vamos a interpretar las posibles soluciones que podemos obtener del sistema $$ p = (I - A)^{-1} d = Ld $$ Para ello tendremos en cuenta que significa que una matriz sea inversible o no y como impacta esto en las soluciones de mi sistema de ecuaciones lineales.
<p style="text-align: left;">
a) \( (I - A) \ \text{es inversible} \)
</p>
Si la matriz (I - A) tiene inversa esto significa que existe la matriz de Leontief, luego existe solucion unica para este sistema.
<p></p>
<p style="text-align: left;">
b) \( (I - A) \ \text{es inversible y d = 0} \)
</p>
En este caso existe matriz de Leontief pero mi sistema resulta en $p = (I - A)^{-1}\cdot0 = L\cdot0 = 0$  Podemos decir que no existe demanda externa en este caso y que la produccion total es destinada a satisfacer la demanda interna.
<p></p>
<p style="text-align: left;">
c) \( (I - A) \ \text{no es inversible} \)
</p>
Al no existir inversa esto significa que no se puede calcular la matriz de Leontief. Esto tiene dos posibles conclusiones:  <p></p> 
  . El sistema no tiene solucion. <p></p> 
  . Existen infinitas soluciones debido que hay dependencia lineal entre las ecuaciones de mi sistema. Esto puede ocurrir debido a una relacion entre sectores que determinan que no hay unica solucion.
    




### Consigna 2 -  

In [ ]:
##Codigo a arreglar, funciona
## pero devuelve cualquier cosa en la posicion a_nn de la matriz U 
##?????

def elim_gaussiana(A):
    cant_op = 0
    m=A.shape[0]
    n=A.shape[1]
    Ac = A.copy()
    
    if m!=n:
        print('Matriz no cuadrada')
        return
    
    ## desde aqui -- CODIGO A COMPLETAR
    C = np.eye(n) - np.eye(n)
        
    for i in range(0,n):
        M = np.eye(n,n)
        for j in range(0,n):
            if i <j:
                piv = Ac[j][i]/Ac[i][i]
                M[j][i] = -piv
                C[j][i] = piv
        Ac = M@Ac        
         

    Ac = Ac + C
    ## hasta aqui
            
    L = np.tril(Ac,-1) + np.eye(A.shape[0]) 
    U = np.triu(Ac)
    
    return L, U, cant_op

---

---